In [1]:
!pip install reverse_geocoder

In [2]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import folium
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMapWithTime

from bs4 import BeautifulSoup

import reverse_geocoder as rg 
import pprint 
import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.simplefilter('ignore')

In [3]:
# %load src/merging geospatial info.py


import pandas as pd
import pickle


rg = pd.read_pickle('data/rg.pkl')

get_ipython().run_line_magic('run', 'src/pipeline')
D = FraudData('data/data.json')
df = D.get_enriched_df()

loc = df[['venue_latitude','venue_longitude','object_id']]
loc = loc[~loc['venue_latitude'].isnull()]

loc2 = loc.join(rg)
loc3 = loc2.drop(['venue_latitude', 'venue_longitude'],axis = 1)
df_geo = df.merge(loc3, how='left', right_on = 'object_id', left_on = 'object_id')





In [4]:
df = df_geo.copy()
df.head(1)

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,ec_month,ec_week,ec_weekday,ec_day,ec_hour,user_event_delta,city_std,admin1,admin2,country_std
0,fraudster_event,1266062400,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,...,1,1,2,6,1,1.188615,Miami Beach,Florida,Miami-Dade County,US


#### Geographical visualization of all the events

In [5]:
default_loc = df['venue_latitude'].mean(), df['venue_longitude'].mean()

def generateBaseMap(default_location=default_loc, default_zoom_start= 3):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map




In [6]:
df_copy = df.copy()
df_copy['count'] = 1
base_map = generateBaseMap()
HeatMap(data=df_copy[['venue_latitude', 'venue_longitude', 'count']].groupby(['venue_latitude', 'venue_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)


base_map.save('images/events_heatmap.html')

In [7]:
df2 = df_copy[~df_copy['venue_latitude'].isnull()]
df2.shape

(13261, 65)

In [8]:
lat = df2['venue_latitude'].tolist()
lon = df2['venue_longitude'].tolist()
locations = list(zip(lat, lon))

map1 = folium.Map(location=default_loc, zoom_start=3)
FastMarkerCluster(data=locations).add_to(map1)
map1.save('images/events_ClusteredMarker.html')

#### Geographical visualization of fraudulent events only

In [9]:
fraud_all = df.query('label == 1')
fraud_all.shape

(1293, 64)

In [10]:
fraud = df2.query('label == 1')
fraud.shape

# 70% of the fraudulent events have geo info.

(923, 65)

In [11]:
fraud['count'] = 1
base_map = generateBaseMap()
HeatMap(data=fraud[['venue_latitude', 'venue_longitude', 'count']].groupby(['venue_latitude', 'venue_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
base_map.save('images/fraud_heatmap.html')

In [12]:
lat = fraud['venue_latitude'].tolist()
lon = fraud['venue_longitude'].tolist()
locations = list(zip(lat, lon))

map2 = folium.Map(location= default_loc, zoom_start= 3)
FastMarkerCluster(data=locations).add_to(map2)
map2.save('images/fraud_ClusteredMarker.html')

#### Geographical Visualization with Popup

#### Heatmap with Time

In [13]:
fraud['ec_mth_yr'] = pd.to_datetime(fraud.ec_year.astype(str) + '-' + fraud.ec_month.astype(str))

In [14]:
fraud.ec_mth_yr.value_counts()

2013-09-01    68
2012-10-01    52
2012-03-01    51
2012-11-01    50
2012-12-01    44
2013-08-01    42
2013-01-01    40
2013-07-01    40
2012-04-01    39
2013-06-01    36
2012-05-01    35
2012-06-01    33
2012-09-01    29
2012-07-01    27
2012-02-01    26
2013-03-01    25
2013-02-01    22
2011-12-01    20
2013-04-01    19
2011-02-01    18
2011-01-01    18
2011-05-01    17
2011-04-01    17
2011-07-01    17
2012-08-01    17
2011-08-01    16
2013-05-01    14
2013-10-01    13
2012-01-01    13
2011-03-01    11
2011-06-01    11
2011-09-01     6
2011-11-01     6
2008-03-01     6
2011-10-01     5
2007-02-01     3
2009-03-01     3
2010-02-01     3
2009-11-01     3
2010-05-01     1
2010-12-01     1
2010-11-01     1
2010-08-01     1
2010-07-01     1
2009-10-01     1
2010-01-01     1
2010-09-01     1
Name: ec_mth_yr, dtype: int64

In [15]:
event_dt_list = []
for dt in fraud.ec_mth_yr.sort_values().unique():
    event_dt_list.append(fraud.loc[fraud.ec_mth_yr == dt, ['venue_latitude', 'venue_longitude', 'count']].groupby(['venue_latitude', 'venue_longitude']).sum().reset_index().values.tolist())

In [16]:
base_map = generateBaseMap()
HeatMapWithTime(event_dt_list, radius=3, gradient={0.8: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map)
base_map.save('images/fraud_heatmap_time.html')

